## Step 1: Data Ingestion
Ingest data from the two provided CSV files. One contains patient details, and the other contains appointment data.

In [1]:
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder \
    .appName("hello_heart_data") \
    .getOrCreate()

# Read the CSV files
appointments_df = spark.read.csv('sample_data/appointment_data.csv', header=True, inferSchema=True)
patients_df = spark.read.csv('sample_data/patient_data.csv', header=True, inferSchema=True)

# Get row counts
print(f"\nNumber of appointments: {appointments_df.count()}")
print(f"Number of patients: {patients_df.count()}")

print("\nSample Appointments:")
appointments_df.show(5, truncate=False)

print("\nSample Patients:")
patients_df.show(50, truncate=False)


Number of appointments: 1000
Number of patients: 1000

Sample Appointments:
+----------+----------------+--------------+
|patient_id|appointment_date|doctor        |
+----------+----------------+--------------+
|236       |2024-05-17      |Morgan Baker  |
|225       |2024-08-03      |Vincent Wright|
|831       |2024-07-19      |Joshua Ford   |
|116       |2024-03-02      |Michelle Hill |
|433       |2024-01-17      |Kari Morse    |
+----------+----------------+--------------+
only showing top 5 rows


Sample Patients:
+----------+------------------+---+-------------------------------------------------------+---------------------+-------------+
|patient_id|name              |age|address                                                |phone_number         |diagnosis    |
+----------+------------------+---+-------------------------------------------------------+---------------------+-------------+
|1         |Nicole Taylor     |36 |6377 Jennifer Trail Apt. 075, Calebside, NY 22906      |

## Step 2: Data Transformation (Should occur before de-identification)
Clean and transform the data. Ensure that:
- Phone numbers and addresses are in a consistent format.
- Data is deduplicated based on patient_id.
- Join the two datasets using patient_id to create a single view of the patient and their appointment history.

In [2]:
from pyspark.sql.functions import regexp_replace, concat_ws, col, when, split, length, count, countDistinct

# Clean the original column to remove non-numeric characters for clarity
patients_df = patients_df.withColumn("phone_number", regexp_replace(col("phone_number"), r"[^0-9x]", ""))

# Split phone_number into core_number and extension
split_col = split(col("phone_number"), "x")
patients_df = patients_df.withColumn("core_number", split_col.getItem(0)) \
                         .withColumn("extension", split_col.getItem(1))

# Strip country code based on the length of core_number
patients_df = patients_df.withColumn(
    "core_number",
    when((length(col("core_number")) == 11), col("core_number").substr(2, 10))
    .when((length(col("core_number")) == 12), col("core_number").substr(3, 10))
    .when((length(col("core_number")) == 13), col("core_number").substr(4, 10))
    .otherwise(col("core_number"))
)

# Recombine core phone number and extension
patients_df = patients_df.withColumn(
    "phone_number",
    when(col("extension") != "", concat_ws("x", col("core_number"), col("extension")))
    .otherwise(col("core_number"))
)

# Drop intermediary columns
patients_df = patients_df.drop("core_number", "extension")

# TODO: MAKE THIS NOT SUCK
# Normalize addresses by removing leading zeroes, extra spaces, and extra commas
patients_df = patients_df.withColumn("address", regexp_replace(col("address"), r"\s+", " ")) \
       .withColumn("address", regexp_replace(col("address"), r",+", ",")) \
       .withColumn("address", regexp_replace(col("address"), r"^0+", ""))
       #.withColumn("address", initcap(trim(col("address")))) \

# Show normalized df
patients_df.show(10, truncate=False)

# No duplicates found in this file, but dupes could be found in future files.
patients_df.select(
    count("patient_id").alias("total_patient_id"),
    countDistinct("patient_id").alias("distinct_patient_id")
).show()

# Drop duplicates based on patient_id
patients_df = patients_df.dropDuplicates(["patient_id"])

# Join patients to appointments and create a view
appointment_history_df = patients_df.join(appointments_df, on="patient_id", how="inner")
appointment_history_df.createOrReplaceTempView("appointment_history")
appointment_history_df.show(10, truncate=False)


+----------+---------------+---+----------------------------------------------------+----------------+------------+
|patient_id|name           |age|address                                             |phone_number    |diagnosis   |
+----------+---------------+---+----------------------------------------------------+----------------+------------+
|1         |Nicole Taylor  |36 |6377 Jennifer Trail Apt. 075, Calebside, NY 22906   |0808983247      |Asthma      |
|2         |Nathan Dorsey  |46 |56 Thompson Park Suite 212, West Anitaport, NY 11534|2826032670x455  |Asthma      |
|3         |Jennifer Garcia|61 |792 Mark Wells, Jaclynport, TN 90027                |8614642617x287  |Diabetes    |
|4         |Joshua Simon   |41 |5655 Harris Inlet, Nealton, MS 89986                |5312690179      |Asthma      |
|5         |Juan Gallegos  |37 |150 Miranda Unions, Bradburgh, FL 64440             |7719585462x30717|Asthma      |
|6         |Dawn Holland   |39 |2523 Flores Radial Suite 462, South Hann

## Step 3: De-identification (Should occur after transformation)
De-identify sensitive patient data such as name, address, and phone_number using an anonymization technique (e.g., hashing).
Ensure that the data can still be linked across the two datasets via a secure common identifier (e.g., patient_id).

In [3]:
from pyspark.sql.functions import sha2

# PII columns to hash
pii_columns_patients = ['name', 'address', 'phone_number']

# Hash PII columns in appointment_history dataframe
appointment_history_anon = appointment_history_df.select(
    *[sha2(col, 256).alias(col) if col in pii_columns_patients 
      else col for col in appointment_history_df.columns]
)

print("\nSample of anonymized patients:")
appointment_history_anon.show(10, truncate=True)

# Store these as new dataframes for further analysis
appointment_history_df = appointment_history_anon
print(appointment_history_df.count())


Sample of anonymized patients:
+----------+--------------------+---+--------------------+--------------------+------------+----------------+------------------+
|patient_id|                name|age|             address|        phone_number|   diagnosis|appointment_date|            doctor|
+----------+--------------------+---+--------------------+--------------------+------------+----------------+------------------+
|         3|9a968bebac0b2fd19...| 61|d9eb2a0b663611b13...|d8f795b1146bd0a98...|    Diabetes|      2024-05-12|  Kristina Collins|
|         4|f19ee33dad5f31ec1...| 41|c0ed2953fbb1e7e53...|fe72aeb66c14c151b...|      Asthma|      2024-08-17|     Felicia Moore|
|         5|12b5e6c34b4527272...| 37|c65ddb9070d9568f1...|6c821939cf5c983be...|      Asthma|      2024-05-08|      Jason Taylor|
|         5|12b5e6c34b4527272...| 37|c65ddb9070d9568f1...|6c821939cf5c983be...|      Asthma|      2024-03-20|  Eugene Wilson MD|
|         6|287946ab5d3399cbd...| 39|916c39d5b4b7d5cb0...|8c55e84

## Step 4: Data Storage
- Set up LocalStack to simulate an S3 environment.
- Store the de-identified and transformed data as Parquet files in the LocalStack S3 bucket.
- Ensure your solution uses Docker to run the pipeline in a containerized environment.

In [4]:
import requests

# Test connection to LocalStack S3 service
url = "http://localstack:4566"
try:
    response = requests.get(url)
    if response.status_code == 200:
        print("Successfully connected to LocalStack:", response.text)
    else:
        print("Failed to connect to LocalStack. Status code:", response.status_code)
except requests.exceptions.RequestException as e:
    print("Error connecting to LocalStack:", e)


Successfully connected to LocalStack: 


In [5]:
import boto3

# Connect to LocalStack S3
s3 = boto3.client(
    "s3",
    endpoint_url="http://localstack:4566",
    aws_access_key_id="test",
    aws_secret_access_key="test",
    region_name="us-east-1"
)

# Create the bucket
bucket_name = "my-test-bucket"
try:
    s3.create_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' created successfully.")
except Exception as e:
    print("Error creating bucket:", e)


Bucket 'my-test-bucket' created successfully.


In [7]:
import shutil
import os

# Local directory where the files are stored
local_temp_path = "/tmp/data/appointment_history.parquet"

# Delete everything in tmp
if os.path.exists("/tmp/data/"):
    shutil.rmtree("/tmp/data/")

# Write appointment history to tmp folder as parquet
appointment_history_df.write.parquet(local_temp_path, mode='overwrite')

# Get the list of parquet part files (excluding any meta files like .crc)
parquet_files = [f for f in os.listdir(local_temp_path) if f.endswith('.parquet')]

# Upload each part file to S3
for part_file in parquet_files:
    part_file_path = os.path.join(local_temp_path, part_file)
    s3_key = f"{part_file}"  # Use a dynamic key, such as a folder with the filename

    # Upload the part file to S3
    s3.upload_file(part_file_path, bucket_name, s3_key)
    print(f"Uploaded {part_file} to S3 bucket '{bucket_name}' with key '{s3_key}'.")

print(f"All part files have been successfully uploaded to S3 bucket '{bucket_name}'.")

Uploaded part-00000-65e94d3f-4118-46a4-a9ce-b79bbae62fe2-c000.snappy.parquet to S3 bucket 'my-test-bucket' with key 'part-00000-65e94d3f-4118-46a4-a9ce-b79bbae62fe2-c000.snappy.parquet'.
All part files have been successfully uploaded to S3 bucket 'my-test-bucket'.


## Step 5: Data Join with PySpark
- Use PySpark to load the two Parquet tables (patient data and appointment data) from the LocalStack S3 bucket.
- Join the two tables on patient_id and print the resulting joined dataset.

In [8]:
# Data is already joined, pulling down from Localstack bucket and displaying...

# List all files in the S3 bucket
response = s3.list_objects_v2(Bucket=bucket_name)

# Filter the list of files to only include Parquet files
parquet_files = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.parquet')]

# Temporary directory to store the files
temp_dir = '/tmp/parquet_files'

# Ensure the directory exists
os.makedirs(temp_dir, exist_ok=True)

# Download each Parquet file from S3 and save it locally
for file_key in parquet_files:
    # Download the file
    file_obj = s3.get_object(Bucket=bucket_name, Key=file_key)
    file_data = file_obj['Body'].read()
    
    # Write the file data to a local file
    local_path = os.path.join(temp_dir, os.path.basename(file_key))
    with open(local_path, 'wb') as f:
        f.write(file_data)

# Now read all the Parquet files from the local directory into a single Spark DataFrame
df = spark.read.parquet(temp_dir)

# Show the DataFrame
df.show()
print(df.count())
df.select("patient_id").where("patient_id = 3").show()

+----------+--------------------+---+--------------------+--------------------+-------------+----------------+------------------+
|patient_id|                name|age|             address|        phone_number|    diagnosis|appointment_date|            doctor|
+----------+--------------------+---+--------------------+--------------------+-------------+----------------+------------------+
|         3|9a968bebac0b2fd19...| 61|d9eb2a0b663611b13...|d8f795b1146bd0a98...|     Diabetes|      2024-05-12|  Kristina Collins|
|         4|f19ee33dad5f31ec1...| 41|c0ed2953fbb1e7e53...|fe72aeb66c14c151b...|       Asthma|      2024-08-17|     Felicia Moore|
|         5|12b5e6c34b4527272...| 37|c65ddb9070d9568f1...|6c821939cf5c983be...|       Asthma|      2024-05-08|      Jason Taylor|
|         5|12b5e6c34b4527272...| 37|c65ddb9070d9568f1...|6c821939cf5c983be...|       Asthma|      2024-03-20|  Eugene Wilson MD|
|         6|287946ab5d3399cbd...| 39|916c39d5b4b7d5cb0...|8c55e84c66abe2cc7...|         No